# Task 1:

## Load a pre-trained GPT-2 model from Hugging Face and generate text based on a prompt.

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
# load a gpt-2 model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# define a prompt
prompt = "Rohit Sharma is best crickter"

In [ ]:
# tokenize the input prompts
input_ids = tokenizer.encode(prompt, return_tensors="pt")

In [ ]:
# generate the text
output = model.generate(
    input_ids,
    max_length=100, # limits the length of generated tokens
    num_return_sequences=1, # species how many responses model should generate
    temperature=0.7, # controls the randomness of text generation (temperature=0.1 more predictive and repetitive, temperature=1.5 more creative but lack coherence)
    top_k=50, # when generating the text, LLMs calculates the probabilites for all possible next words, it specifies only top k words will be considered for selection
    top_p=0.9, # selects smallest set of words whose cummulative probability adds up to p
    do_sample=True # enables randomness in text generation
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
# decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Rohit Sharma is best crickter in the world.

It is easy to forget that the Indian cricketers have been known to be at their best during the last few years. They have never been afraid to take risks, to get off on the wrong foot, and to take risks that are in the face of overwhelming odds. In the last two years, they have been the top cricketers in the world. They are the best cricketers in the world, and


# Task 2:

## Fine-tune a small BERT model for a classification task (eg. sentiment analysis) on a dataset like IMDB reviews.

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
from transformers import TrainingArguments

In [ ]:
# load IMDB dataset
dataset = load_dataset("imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# load tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

In [ ]:
# tokenize the dataset
tokenized_dataset = dataset.map(tokenization_functions, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
# convert labels and tokenize inputs
tokenized_datasets = tokenized_dataset.rename_column("label", "labels")
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
# split datasets into train and test sets
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

In [ ]:
# load pre-trained small bert model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# setup training arguments
training_args = TrainingArguments(
    output_dir="./bert_sentiment", # saves model checkpoint here
    evaluation_strategy="epoch", # evaluate model after each epoch
    save_strategy="epoch", # saves the model after every epoch
    per_device_train_batch_size=16, # uses 8 training samples per gpu
    per_device_eval_batch_size=16, # controls the batch size during evaluation
    num_train_epochs=3,
    fp16=True,
    logging_steps=500,
    load_best_model_at_end=True

)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# define trainer and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.231600,0.250983
2,0.123800,0.224394
3,0.056700,0.296085


TrainOutput(global_step=4689, training_loss=0.1545454063098246, metrics={'train_runtime': 1253.5207, 'train_samples_per_second': 59.831, 'train_steps_per_second': 3.741, 'total_flos': 9935054899200000.0, 'train_loss': 0.1545454063098246, 'epoch': 3.0})

In [ ]:
# Define evaluation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Run evaluation
trainer.compute_metrics = compute_metrics
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.22439439594745636, 'eval_accuracy': 0.92464, 'eval_runtime': 93.1334, 'eval_samples_per_second': 268.432, 'eval_steps_per_second': 16.782, 'epoch': 3.0}


In [ ]:
# Detect available device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the detected device
model.to(device)

# Prediction function
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits).item()
    return "Positive" if prediction == 1 else "Negative"

# Test example
review = "This movie was worse! I hated the performances and the story."
print(predict_sentiment(review))

Negative


# Task 3:

## Experiment with prompt engineering. Compare the outputs of the model (e.g. GPT-2) with different prompts for same task (e.g. summarizing a paragrapgh).

In [1]:
from transformers import pipeline

In [2]:
# load GPT-2 model
generator = pipeline("text-generation", model="gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
# sample paragraph for summarization
sample_paragraph = (
    "Artificial intelligence (AI) is transforming industries worldwide. "
    "With advancements in machine learning, AI systems can now analyze large datasets, "
    "recognize patterns, and make decisions faster than humans. Companies are increasingly "
    "using AI to improve customer experiences, automate tasks, and enhance decision-making. "
    "However, concerns about data privacy and job displacement remain significant challenges as AI continues to evolve."
)

In [8]:
# prompt1: direct instruction prompt
prompt1 = "Summarize the following paragraph:\n" + sample_paragraph
output1 = generator(prompt1, max_length=100, num_return_sequences=1)[0]['generated_text']
print("\n🔹 Direct Instruction Output:\n", output1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🔹 Direct Instruction Output:
 Summarize the following paragraph:
Artificial intelligence (AI) is transforming industries worldwide. With advancements in machine learning, AI systems can now analyze large datasets, recognize patterns, and make decisions faster than humans. Companies are increasingly using AI to improve customer experiences, automate tasks, and enhance decision-making. However, concerns about data privacy and job displacement remain significant challenges as AI continues to evolve.

The main issue facing AI is the ability to predict future human behavior, but what these systems


In [12]:
# prompt2: role-based prompt
prompt2 = "You are an expert summarizer. Summarize the following text in simple words:\n" + sample_paragraph
output2 = generator(prompt2, max_length=100, num_return_sequences=1)[0]['generated_text']
print("\n🔹 Role-Based Output:\n", output2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🔹 Role-Based Output:
 You are an expert summarizer. Summarize the following text in simple words:
Artificial intelligence (AI) is transforming industries worldwide. With advancements in machine learning, AI systems can now analyze large datasets, recognize patterns, and make decisions faster than humans. Companies are increasingly using AI to improve customer experiences, automate tasks, and enhance decision-making. However, concerns about data privacy and job displacement remain significant challenges as AI continues to evolve. According to a recent report in the Wall Street Journal


In [15]:
# prompt3: few-shot prompting
prompt3 = (
    "Summarize the following text in one sentence:\n"
    "Text: The cat is sleeping under the tree. \nSummary: The cat is resting. \n---\nText: "
    + sample_paragraph
)
output3 = generator(prompt3, max_length=200, num_return_sequences=1)[0]['generated_text']
print("\n🔹 Few-Shot Prompt Output:\n", output3)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🔹 Few-Shot Prompt Output:
 Summarize the following text in one sentence:
Text: The cat is sleeping under the tree. 
Summary: The cat is resting. 
---
Text: Artificial intelligence (AI) is transforming industries worldwide. With advancements in machine learning, AI systems can now analyze large datasets, recognize patterns, and make decisions faster than humans. Companies are increasingly using AI to improve customer experiences, automate tasks, and enhance decision-making. However, concerns about data privacy and job displacement remain significant challenges as AI continues to evolve. While AI is improving customer experiences or providing superior product opportunities, companies are facing a serious economic, technological, and social disruption. Because of these challenges, businesses must address what matters most to them: how they generate the value of product, service, employee and technology opportunities, where the data comes from, how they interact with and manage the public

In [19]:
# prompt4: instruction with length constraint
prompt4 = "Summarize this paragraph in 2 sentences:\n" + sample_paragraph
output4 = generator(prompt4, max_length=150, num_return_sequences=1)[0]['generated_text']
print("\n🔹 Instruction with Length Constraint Output:\n", output4)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🔹 Instruction with Length Constraint Output:
 Summarize this paragraph in 2 sentences:
Artificial intelligence (AI) is transforming industries worldwide. With advancements in machine learning, AI systems can now analyze large datasets, recognize patterns, and make decisions faster than humans. Companies are increasingly using AI to improve customer experiences, automate tasks, and enhance decision-making. However, concerns about data privacy and job displacement remain significant challenges as AI continues to evolve.

4. "Human-Computer Interaction" and "AI vs. Human"

2) Artificial intelligence will change human relations at the local, state, and national level, and at the digital level

In the coming years, the next phase of AI, "AI vs. Human," will be determined more by economic


In [22]:
# prompt5: keyword based prompt
prompt5 = "Summarize the following focusing on AI's impact on jobs and privacy:\n" + sample_paragraph
output5 = generator(prompt5, max_length=150, num_return_sequences=1)[0]['generated_text']
print("\n🔹 Keyword-Based Prompt Output:\n", output5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🔹 Keyword-Based Prompt Output:
 Summarize the following focusing on AI's impact on jobs and privacy:
Artificial intelligence (AI) is transforming industries worldwide. With advancements in machine learning, AI systems can now analyze large datasets, recognize patterns, and make decisions faster than humans. Companies are increasingly using AI to improve customer experiences, automate tasks, and enhance decision-making. However, concerns about data privacy and job displacement remain significant challenges as AI continues to evolve.

Artificial intelligence (AI) is transforming industries worldwide. With advancements in machine learning, AI systems can now analyze large datasets, recognize patterns, and make decisions faster than humans. Companies are increasingly using AI to improve customer experiences, automate tasks, and enhance decision-making. However, concerns about data privacy
